In [2]:
%load_ext sql

Loading configurations from /home/hugo/Documents/skola/edaf75/repo/pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True


In [3]:
%sql sqlite:///movies.sqlite

Connecting to 'sqlite:///movies.sqlite'

In [9]:
%%sql
select *
from performances

Running query in 'sqlite:///movies.sqlite'

,performance_id,start_time,date,theater_name,imdb
0,7a66e300e5e2304f18a6c4a4a7584c5a,19:00,2021-02-22,Regal,tt3661210
1,61f28a723c476f94d10252a8aaad403d,21:00,2021-02-22,Regal,tt3661210
2,a32f9fd9ce0cd18f1e722cd0c3bc06e6,19:00,2021-02-22,Skandia,tt10016180


In [21]:
%%sql
insert
into   tickets(user_name, performance_id)
values ('bob', 'a32f9fd9ce0cd18f1e722cd0c3bc06e6')
returning ticket_id

Running query in 'sqlite:///movies.sqlite'

,ticket_id
0,3a0d4f1fb9918e8ed0a9bcdd95bd3d05


In [14]:
%%sql
select *
from tickets

Running query in 'sqlite:///movies.sqlite'

,ticket_id,user_name,performance_id
0,69c3af98393e9819ae537164c65d875c,bob,7a66e300e5e2304f18a6c4a4a7584c5a
1,3583edacee13bc2a0a9b21e3acbfb362,bob,7a66e300e5e2304f18a6c4a4a7584c5a


In [28]:
%%sql
with taken_seats(performance_id, taken) as (
    select performance_id, count() as taken
    from performances
            join theaters using(theater_name)
            join tickets using(performance_id)
    group by performance_id
)
select performance_id, date, start_time, title, year, theater_name, capacity - coalesce(taken, 0) as remaining_seats
from performances
        join theaters using(theater_name)
        join movies using(imdb)
     left outer join taken_seats using(performance_id)

Running query in 'sqlite:///movies.sqlite'

,performance_id,date,start_time,title,year,theater_name,remaining_seats
0,5fa0059c1b18646d54cf4204d2ac6109,2021-02-22,19:00,TheDig,2021,Regal,14
1,fea61d39de2410b6d14a0481166ef26e,2021-02-22,21:00,TheDig,2021,Regal,16
2,7e6b684bcc6a056689e88b1d3ec6d9f7,2021-02-22,19:00,TheLittleThings,2021,Skandia,100


In [22]:
%%sql
with user_tickets(performance_id, count) as (
    select   performance_id, count() as count
    from     tickets
    where    user_name = 'bob'
    group by performance_id
)
select date, start_time, theater_name, title, year, count as nbrOfTickets
from   user_tickets
        join performances using(performance_id)
        join movies using(imdb)

Running query in 'sqlite:///movies.sqlite'

,date,start_time,theater_name,title,year,nbrOfTickets
0,2021-02-22,19:00,Regal,TheDig,2021,2
1,2021-02-22,19:00,Skandia,TheLittleThings,2021,1
